In [ ]:
!pip install pandas

In [ ]:
!pip install sklearn

In [ ]:
!pip install matplotlib

In [ ]:
!pip install librosa==0.8.0

In [ ]:
!apt-get update

In [ ]:
!apt-get install libsndfile-dev -y

In [ ]:
import tensorflow as tf


In [ ]:
!pip install keras

In [ ]:
from models import STN

In [ ]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)

    except RuntimeError as e:
        print(e)

In [ ]:
import pandas as pd
import os
from matplotlib import pyplot as plt
import numpy as np
from pathlib import Path
import tensorflow as tf
from tensorflow.keras.mixed_precision import experimental as mixed_precision
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, Callback
from tensorflow.keras.models import load_model

In [ ]:
batch_size = 64
num_epochs = 30
target_size = (256, 256)
color_mode = 'grayscale'
dataset = '/raid/audio_detection/audio_detection/data'
dataset_1= 'new'
train_datagen = ImageDataGenerator(rescale=1/255)
val_datagen = ImageDataGenerator(rescale=1/255)
test_datagen = ImageDataGenerator(rescale=1/255)
print(os.getcwd())
print(os.chdir(dataset))
print(os.getcwd())


train_generator = train_datagen.flow_from_directory(f'{dataset}/train/', target_size=target_size, batch_size=batch_size, class_mode='binary', color_mode=color_mode, classes = ['real','fake'])
val_generator = val_datagen.flow_from_directory(f'{dataset}/val/', target_size=target_size, batch_size=batch_size, class_mode='binary', color_mode=color_mode, classes = ['real','fake'])
test_generator = test_datagen.flow_from_directory(f'{dataset}/test/', target_size=target_size, batch_size=batch_size, class_mode='binary', color_mode=color_mode, classes = ['real','fake'])


In [ ]:
from keras.layers import Input, Activation, Dense, Conv1D, Dropout, BatchNormalization, Reshape

In [ ]:
model = STN()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
from models.utils import get_initial_weights
from models.layers import BilinearInterpolation

In [ ]:
for attempt in range(1):
    print(os.getcwd())
    model_path = f'models/{dataset}_{attempt}'
    print(os.getcwd())
    Path(model_path).mkdir(parents=True, exist_ok = True)
    checkpointer = ModelCheckpoint(model_path +'tn_new_JANAVI_epochs:{epoch:02d}_acc:{val_accuracy:.4f}.h5', monitor = 'val_loss', save_best_only = True, verbose = 0, mode = 'auto')
    #checkpointer = ModelCheckpoint(filepath='model._epochs:{epoch:02d}_acc:{val_accuracy:.4f}.h5',monitor = 'val_loss', save_best_only = True, verbose = 0, mode = 'auto')
    earlystopper = EarlyStopping(monitor = 'val_loss', patience = 5, verbose = 0, mode = 'auto')
    reduceLR = ReduceLROnPlateau(monitor = 'val_loss', factor = 1/np.sqrt(10), patience = 3, cooldown = 1, verbose = 0, mode = 'auto')
    model = STN()
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    history = model.fit(train_generator, steps_per_epoch = len(train_generator), verbose = 2, epochs = 10, callbacks = [checkpointer, earlystopper, reduceLR], validation_data=val_generator, validation_steps = len(val_generator))
    print(model.evaluate(test_generator, steps = 2*len(test_generator)))
    
    ## added later on
    results = model.evaluate(val_generator,steps=2*len(val_generator),verbose=0)
    print('Development Set:')
    print(f'Loss: {results[0]:.4f}\nAccuracy: {results[1]:.4f}')
    results = model.evaluate(test_generator,steps=2*len(test_generator),verbose=0)
    print('Evaluation Set:')
    print(f'Loss: {results[0]:.4f}\nAccuracy: {results[1]: .4f}')

In [ ]:
#model = load_model("/raid/audio_detection/audio_detection/data/models/raid/audio_detection/audio_detection/data_0tn_new_epochs:01_acc:0.7894.h5",custom_objects={'BilinearInterpolation': BilinearInterpolation})

In [ ]:
!pip install pillow

In [ ]:
from PIL import Image

In [ ]:
x,y = next(test_generator)

In [ ]:
predictions = model.predict(x)
predictions

In [ ]:
for i in predictions:
    if i[0] < 0.5:
        i[0] = 0.0
    else :
        i[0] = 1.0

In [ ]:
!pip install scikit-learn

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [ ]:
matrix = confusion_matrix(y,predictions, labels=[1,0])

In [ ]:
print(matrix)

In [ ]:
matrix = classification_report(y,predictions,labels=[1,0])
print('Classification report : \n',matrix)